In [43]:
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp

In [44]:
def create_data_model():
    """Stores the data for the problem."""
    data = {}
    with open("time_matrix.txt", "r") as f:
        data["time_matrix"] = [
            [int(num) for num in line.split("\t")] for line in f.readlines()]
    data['time_windows'] = [
        (0, 600),   # depot
        (800, 1100),  # 1
        (1000, 1300),  # 2
        (400, 750),   # 3
        (850, 1200),  # 4
        (600, 1000),  # 5
        (850, 1100),  # 6
        (500, 900),   # 7
        (1200, 1400),  # 8
        (400, 800),   # 9
        (550, 850),   # 10
        (150, 1000),  # 11
        (600, 1500),  # 12
        (700, 1200),  # 13
        (900, 1300),  # 14
        (250, 700),   # 15
        (400, 1000),  # 16
    ]
    data['demands'] = [0, 4, 5, 9, 5, 5, 3,
                       3, 8, 4, 8, 1, 6, 3, 2, 7, 2]   # weights/demands for each node
    data['vehicle_capacities'] = [20, 20, 5, 20, 20, 20, 10, 20, 20, 20]
    data['num_vehicles'] = 10
    data['depot'] = 0
    return data

In [45]:
def print_solution(data, manager, routing, solution):
    """Prints solution on console."""
    print(f'Objective: {solution.ObjectiveValue()}')
    time_dimension = routing.GetDimensionOrDie('Time')
    total_time = 0
    total_load = 0
    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        plan_output = 'Route for vehicle {}:\n'.format(vehicle_id)
        route_load = 0
        while not routing.IsEnd(index):
            time_var = time_dimension.CumulVar(index)
            node_index = manager.IndexToNode(index)
            route_load += data['demands'][node_index]
            plan_output += '{0} Time({1},{2}) Load({3}) -> '.format(
                node_index, solution.Min(time_var),
                solution.Max(time_var), route_load)
            index = solution.Value(routing.NextVar(index))
        time_var = time_dimension.CumulVar(index)
        node_index = manager.IndexToNode(index)
        route_load += data['demands'][node_index]
        plan_output += '{0} Time({1},{2}) Load({3})\n'.format(node_index,
                                                              solution.Min(
                                                                  time_var),
                                                              solution.Max(
                                                                  time_var),
                                                              route_load)
        plan_output += 'Time of the route: {}min\n'.format(
            solution.Min(time_var))
        plan_output += 'Load of the route: {}\n'.format(route_load)
        print(plan_output)
        total_time += solution.Min(time_var)
    print('Total time of all routes: {}min'.format(total_time))

In [46]:
"""Solve the VRP with time windows."""
# Instantiate the data problem.
data = create_data_model()

In [47]:
# Create the routing index manager
# The inputs to RoutingIndexManager are:
#   The number of locations (including the depot)
#   The number of vehicles in the problem
#   The node corresponding to the depot

manager = pywrapcp.RoutingIndexManager(
    len(data['time_matrix']), data['num_vehicles'], data['depot'])

In [48]:
# Create Routing Model.
routing = pywrapcp.RoutingModel(manager)

In [49]:
# Create and register a transit callback.
def time_callback(from_index, to_index):
    """Returns the travel time between the two nodes."""
    # Convert from routing variable Index to time matrix NodeIndex.
    from_node = manager.IndexToNode(from_index)
    to_node = manager.IndexToNode(to_index)
    return data['time_matrix'][from_node][to_node]


transit_callback_index = routing.RegisterTransitCallback(time_callback)

In [50]:
# Define cost of each arc.
routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

In [51]:
# Add Time Windows constraint.
import math
time = 'Time'
routing.AddDimension(
    transit_callback_index,
    1500,  # allow waiting time (1440 as it means the whole day in minutes)
    2500,  # maximum time per vehicle (all times are assumed to be in minutes)
    False,  # Don't force start cumul to zero.
    time)
time_dimension = routing.GetDimensionOrDie(time)
# Add time window constraints for each location except depot.
for location_idx, time_window in enumerate(data['time_windows']):
    if location_idx == data['depot']:
        continue
    index = manager.NodeToIndex(location_idx)
    time_dimension.CumulVar(index).SetRange(time_window[0], time_window[1])
# Add time window constraints for each vehicle start node.
depot_idx = data['depot']
for vehicle_id in range(data['num_vehicles']):
    index = routing.Start(vehicle_id)
    time_dimension.CumulVar(index).SetRange(
        data['time_windows'][depot_idx][0],
        data['time_windows'][depot_idx][1])

In [52]:
# Instantiate route start and end times to produce feasible times.
for i in range(data['num_vehicles']):
    routing.AddVariableMinimizedByFinalizer(
        time_dimension.CumulVar(routing.Start(i)))
    routing.AddVariableMinimizedByFinalizer(
        time_dimension.CumulVar(routing.End(i)))

In [53]:
# Add Capacity constraint.
def demand_callback(from_index):
    """Returns the demand of the node."""
    # Convert from routing variable Index to demands NodeIndex.
    from_node = manager.IndexToNode(from_index)
    return data['demands'][from_node]


demand_callback_index = routing.RegisterUnaryTransitCallback(
    demand_callback)
routing.AddDimensionWithVehicleCapacity(
    demand_callback_index,
    0,  # null capacity slack
    data['vehicle_capacities'],  # vehicle maximum capacities
    True,  # start cumul to zero
    'Capacity')

True

In [54]:
# Setting first solution heuristic.
search_parameters = pywrapcp.DefaultRoutingSearchParameters()
search_parameters.first_solution_strategy = (
    routing_enums_pb2.FirstSolutionStrategy.PARALLEL_CHEAPEST_INSERTION)

In [55]:
# Solve the problem.
import time
start_time = time.time()
solution = routing.SolveWithParameters(search_parameters)
end_time = time.time()
print(end_time-start_time)

0.023087739944458008


In [56]:
# Print solution on console.
if solution:
    print_solution(data, manager, routing, solution)
else:
    print("No solution found !")

Objective: 16144
Route for vehicle 0:
0 Time(0,0) Load(0) -> 9 Time(447,447) Load(4) -> 1 Time(878,878) Load(8) -> 8 Time(1263,1263) Load(16) -> 0 Time(2271,2271) Load(16)
Time of the route: 2271min
Load of the route: 16

Route for vehicle 1:
0 Time(0,0) Load(0) -> 6 Time(850,850) Load(3) -> 0 Time(1468,1468) Load(3)
Time of the route: 1468min
Load of the route: 3

Route for vehicle 2:
0 Time(0,0) Load(0) -> 11 Time(627,627) Load(1) -> 13 Time(1196,1196) Load(4) -> 0 Time(2334,2334) Load(4)
Time of the route: 2334min
Load of the route: 4

Route for vehicle 3:
0 Time(0,0) Load(0) -> 3 Time(533,533) Load(9) -> 16 Time(996,996) Load(11) -> 12 Time(1319,1319) Load(17) -> 0 Time(2471,2471) Load(17)
Time of the route: 2471min
Load of the route: 17

Route for vehicle 4:
0 Time(0,0) Load(0) -> 15 Time(526,659) Load(7) -> 2 Time(1000,1000) Load(12) -> 0 Time(1704,1704) Load(12)
Time of the route: 1704min
Load of the route: 12

Route for vehicle 5:
0 Time(0,0) Load(0) -> 7 Time(533,533) Load(3) 